In [8]:
import numpy as np
import numpy.linalg as lg
from scipy.sparse.linalg import qmr
from tabulate import tabulate
from time import time
from  scipy.io import mmread
A1=np.array(mmread('/content/west0067.mtx.gz').todense())
A2=np.array(mmread('/content/jpwh_991.mtx.gz').todense())
A5=np.array(mmread('/content/impcol_a.mtx.gz').todense())
A7=np.array(mmread('/content/impcol_e.mtx.gz').todense())
A8=np.array(mmread('/content/sherman4.mtx.gz').todense())
A10=np.array(mmread('/content/orsreg_1.mtx.gz').todense())
A11=np.array(mmread('/content/orsirr_1.mtx.gz').todense())
A12=np.array(mmread('/content/add20.mtx.gz').todense())
A13=np.array(mmread('/content/gre__185.mtx.gz').todense())
A14=np.array(mmread('/content/fs_680_3.mtx.gz').todense())
A15=np.array(mmread('/content/fs_680_2.mtx.gz').todense())
B1=np.array(mmread('/content/west0381.mtx.gz').todense())
B2=np.array(mmread('/content/e05r0200.mtx.gz').todense())

# The Quasi Minimal Residual algorithm

In [2]:
def qmr_iter(A, b,x0,maxiter,tol):
    num_iters = 0

    def callback(xk):
        nonlocal num_iters
        num_iters += 1
    return qmr(A, b, callback=callback,tol = tol,maxiter=maxiter),num_iters




def QMR(A, b, x,m, tol,Itermax):
    r      = b-np.dot(A,x)
    n      = A.shape[0] 
    beta   = np.linalg.norm(r)
    V      = np.zeros((n,m+2))
    W      = np.zeros((n,m+2))
    V[:,1] = (r/beta) 
    W[:,1] = (r/beta)
    alphaH = np.zeros(m+1) 
    betaH  = np.zeros(m+2)                                                        
    deltaH = np.zeros(m+2)
    
    k  = 0
    while k < Itermax and beta>tol:   
        for j in range(1,m+1):
            alphaH[j]  = np.dot(np.dot(A,V[:,j]),W[:,j])
            vtelda     = np.dot(A,V[:,j])- alphaH[j]*V[:,j]- betaH[j]*V[:,j-1]
            wtelda     = np.dot(A.T,W[:,j])- alphaH[j]*W[:,j]- deltaH[j]*W[:,j-1]
            deltaH[j+1]= abs(np.dot(vtelda ,wtelda))**0.5
            if deltaH[j+1]== 0:
                break
            else:
                betaH[j+1] = np.dot(vtelda ,wtelda)/deltaH[j+1]
                W[:,j+1]   = wtelda/betaH[j+1]
                V[:,j+1]   = vtelda/deltaH[j+1]
        H      = np.diag(alphaH[1:m+1])+np.diag(betaH[2:m+1],1)+np.diag(deltaH[2:m+1],-1)
        em     = np.zeros(m)
        em[m-1]  =1
        Htelda = np.zeros((m+1,m))
        Htelda[:m,:]= H
        Htelda[m:m+1,:]  = deltaH[-1]*em
        #The QMR Algorithm for Linear Systems
        e1   = np.zeros(m+1)
        e1[0]= 1
        Vm   = V[:,1:m+1]
        Q, R = np.linalg.qr(Htelda,mode='complete') 
        Rm   = R[:m,:m] 
        g    = beta*np.dot(np.transpose(Q),e1)
        gm   = g[:m]
        ym   = np.linalg.solve(Rm, gm)
        # l'itération  de la méthode
        x    = x+np.dot(Vm,ym)
        r    = b - np.dot(A, x)
        beta = np.linalg.norm(r)
        V[:, 1] = r/beta 
        W[:,1]  = r/beta 
        k     = k + 1  
    return x,k 
    

# Test

In [3]:
n=100
D1 = 2*np.eye(n) +np.diag(np.ones(n-1),-1)-np.diag(np.ones(n-1),1)
print('codition umber :',lg.cond(D1))


codition umber : 1.4137006160045253


In [4]:

b = np.ones(n)
x = np.ones(n)
m = 20
tol=1.e-5
Itermax  = n
xx,k11 = QMR(D1, b, x,m,tol, Itermax)
print('nombre d iterations',k11)
sol=np.linalg.solve(D1, b)
err=np.linalg.norm(xx-sol)
print("\n L'erreur de la méthode: \t ", err) 
y = qmr(D1,b)[0]
print('scipy error',np.linalg.norm(y-sol))

nombre d iterations 1

 L'erreur de la méthode: 	  2.595966789165717e-08
scipy error 1.9835423653078597e-05


In [5]:
def qmr_iter(D1, b,x0,maxiter,tol):
    num_iters = 0

    def callback(xk):
        nonlocal num_iters
        num_iters += 1
    return qmr(D1, b, callback=callback,tol = tol,maxiter=maxiter),num_iters

In [6]:
yy = qmr_iter(D1,b,x,Itermax,tol)
print('scipy')
print('le nombre d iteration est',yy[1])
print('le residu',lg.norm(np.dot(D1,yy[0][0])-b))

scipy
le nombre d iteration est 12
le residu 4.581122535614598e-05


# **Cluster 1:**

In [10]:
matricies=[A2,A8,A10,A12,A11,A15,A14]
tol=1e-5
itermax = 10000
m=20

In [11]:
my_data = []
for i in range(len(matricies)):
  l = []
  n = len(matricies[i])
  b=np.ones(n)
  x = np.zeros(n)
  itersc = qmr_iter(matricies[i],b,x,itermax,tol)[1] 
  t3 = time()
  y=qmr(matricies[i],b,x,tol,itermax)[0]
  t4 = time()
  sc_tmp = (t4-t3)*1000      # scipy time in ms 
  sc_res=lg.norm(np.dot(matricies[i],y)-b)
  t1 = time()
  x,iterm= QMR(matricies[i], b, x,m, tol,itermax)
  t2 = time()
  tmp = (t2-t1)*1000      # Our time in ms 
  res=lg.norm(np.dot(matricies[i],x)-b)
  l.append(matricies[i].shape)
  l.append(lg.cond(matricies[i]))
  l.append(iterm)
  l.append(tmp)
  l.append(res)
  l.append(itersc)
  l.append(sc_tmp)
  l.append(sc_res)
  my_data.append(l)
# create header
head = ['matricies','condition number','Our_nbr_iter','time','res','scipy_nbr_iter','sc_time','sc_res']    
# display table
print(tabulate(my_data, headers=head, tablefmt="grid")) 

+--------------+--------------------+----------------+------------+-------------+------------------+-----------+---------------+
| matricies    |   condition number |   Our_nbr_iter |       time |         res |   scipy_nbr_iter |   sc_time |        sc_res |
+==============+====================+================+============+=============+==================+===========+===============+
| (991, 991)   |      142.045       |              3 |    84.3449 | 2.82825e-06 |               40 |   36.1195 |   0.00023833  |
+--------------+--------------------+----------------+------------+-------------+------------------+-----------+---------------+
| (1104, 1104) |     2178.63        |             36 |  1109.68   | 9.26386e-06 |              102 |  102.309  |   0.000125855 |
+--------------+--------------------+----------------+------------+-------------+------------------+-----------+---------------+
| (2205, 2205) |     6745.27        |             17 |  4117.5    | 8.69697e-06 |              12

**Size of Krylov subspace: 30**

In [12]:
matricies=[A2,A8,A10,A12,A11,A15,A14]
tol=1e-5
itermax = 10000
m=30

In [13]:
my_data = []
for i in range(len(matricies)):
  l = []
  n = len(matricies[i])
  b=np.ones(n)
  x = np.zeros(n)
  itersc = qmr_iter(matricies[i],b,x,itermax,tol)[1] 
  t3 = time()
  y=qmr(matricies[i],b,x,tol,itermax)[0]
  t4 = time()
  sc_tmp = (t4-t3)*1000      # scipy time in ms 
  sc_res=lg.norm(np.dot(matricies[i],y)-b)
  t1 = time()
  x,iterm= QMR(matricies[i], b, x,m, tol,itermax)
  t2 = time()
  tmp = (t2-t1)*1000      # Our time in ms 
  res=lg.norm(np.dot(matricies[i],x)-b)
  l.append(matricies[i].shape)
  l.append(lg.cond(matricies[i]))
  l.append(iterm)
  l.append(tmp)
  l.append(res)
  l.append(itersc)
  l.append(sc_tmp)
  l.append(sc_res)
  my_data.append(l)
# create header
head = ['matricies','condition number','Our_nbr_iter','time','res','scipy_nbr_iter','sc_time','sc_res']    
# display table
print(tabulate(my_data, headers=head, tablefmt="grid")) 

+--------------+--------------------+----------------+------------+-------------+------------------+-----------+---------------+
| matricies    |   condition number |   Our_nbr_iter |       time |         res |   scipy_nbr_iter |   sc_time |        sc_res |
+==============+====================+================+============+=============+==================+===========+===============+
| (991, 991)   |      142.045       |              2 |    71.7578 | 1.70884e-06 |               40 |   39.3679 |   0.00023833  |
+--------------+--------------------+----------------+------------+-------------+------------------+-----------+---------------+
| (1104, 1104) |     2178.63        |             14 |   630.386  | 5.40375e-06 |              102 |  134.077  |   0.000125855 |
+--------------+--------------------+----------------+------------+-------------+------------------+-----------+---------------+
| (2205, 2205) |     6745.27        |             11 |  3835.71   | 9.51469e-06 |              12

# **Cluster 2:**

In [14]:
matricies=[A1,A13,B2]
tol=1e-5
itermax = 1000
m=20

In [15]:
my_data = []
for i in range(len(matricies)):
  l = []
  n = len(matricies[i])
  b=np.ones(n)
  x = np.zeros(n)
  itersc = qmr_iter(matricies[i],b,x,itermax,tol)[1] 
  t3 = time()
  y=qmr(matricies[i],b,x,tol,itermax)[0]
  t4 = time()
  sc_tmp = (t4-t3)*1000      # scipy time in ms 
  sc_res=lg.norm(np.dot(matricies[i],y)-b)
  t1 = time()
  x,iterm= QMR(matricies[i], b, x,m, tol,itermax)
  t2 = time()
  tmp = (t2-t1)*1000      # Our time in ms 
  res=lg.norm(np.dot(matricies[i],x)-b)
  l.append(matricies[i].shape)
  l.append(lg.cond(matricies[i]))
  l.append(iterm)
  l.append(tmp)
  l.append(res)
  l.append(itersc)
  l.append(sc_tmp)
  l.append(sc_res)
  my_data.append(l)
# create header
head = ['matricies','condition number','Our_nbr_iter','time','res','scipy_nbr_iter','sc_time','sc_res']    
# display table
print(tabulate(my_data, headers=head, tablefmt="grid")) 

+-------------+--------------------+----------------+---------+-----------------+------------------+-----------+-------------+
| matricies   |   condition number |   Our_nbr_iter |    time |             res |   scipy_nbr_iter |   sc_time |      sc_res |
+=============+====================+================+=========+=================+==================+===========+=============+
| (67, 67)    |            130.217 |           1000 | 1565.21 |     4.7464e+144 |              122 |   14.7717 | 7.44958e-05 |
+-------------+--------------------+----------------+---------+-----------------+------------------+-----------+-------------+
| (185, 185)  |         181227     |           1000 | 2255.92 |     9.98805e+34 |              290 |   39.314  | 0.00013313  |
+-------------+--------------------+----------------+---------+-----------------+------------------+-----------+-------------+
| (236, 236)  |          71941.1   |           1000 | 2355.12 | 23964.1         |              997 |  146.026  

# **Cluster 3:**

In [16]:
matricies=[B1,A7,A5]
tol=1e-5
itermax = 1000
m=20

In [17]:
my_data = []
for i in range(len(matricies)):
  l = []
  n = len(matricies[i])
  b=np.ones(n)
  x = np.zeros(n)
  itersc = qmr_iter(matricies[i],b,x,itermax,tol)[1] 
  t3 = time()
  y=qmr(matricies[i],b,x,tol,itermax)[0]
  t4 = time()
  sc_tmp = (t4-t3)*1000      # scipy time in ms 
  sc_res=lg.norm(np.dot(matricies[i],y)-b)
  t1 = time()
  x,iterm= QMR(matricies[i], b, x,m, tol,itermax)
  t2 = time()
  tmp = (t2-t1)*1000      # Our time in ms 
  res=lg.norm(np.dot(matricies[i],x)-b)
  l.append(matricies[i].shape)
  l.append(lg.cond(matricies[i]))
  l.append(iterm)
  l.append(tmp)
  l.append(res)
  l.append(itersc)
  l.append(sc_tmp)
  l.append(sc_res)
  my_data.append(l)
# create header
head = ['matricies','condition number','Our_nbr_iter','time','res','scipy_nbr_iter','sc_time','sc_res']    
# display table
print(tabulate(my_data, headers=head, tablefmt="grid")) 

+-------------+--------------------+----------------+---------+-------------+------------------+-----------+----------+
| matricies   |   condition number |   Our_nbr_iter |    time |         res |   scipy_nbr_iter |   sc_time |   sc_res |
+=============+====================+================+=========+=============+==================+===========+==========+
| (381, 381)  |        1.25539e+06 |           1000 | 4563.23 | 2.73872e+78 |             1000 |   236.679 |  35.6114 |
+-------------+--------------------+----------------+---------+-------------+------------------+-----------+----------+
| (225, 225)  |        7.10097e+06 |           1000 | 2322.64 | 3.08488e+41 |             1000 |   138.431 |  23.9848 |
+-------------+--------------------+----------------+---------+-------------+------------------+-----------+----------+
| (207, 207)  |        1.35164e+08 |           1000 | 2248.65 | 5.08754e+38 |             1000 |   173.491 |  14.699  |
+-------------+--------------------+----